Part 1

In [15]:
import pandas as pd
import numpy as np

df=pd.DataFrame(pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0])

#Ignore cells with a borough that is Not assigned.
df=df[df['Borough']!='Not assigned']
df.reset_index(inplace=True)
#other changes aren 't neccesary with actual wikipedia table
df.head(11)

(103, 4)

In [ ]:
df.shape

Part 2

In [18]:
#load location from csv
geospatialdata= pd.read_csv('http://cocl.us/Geospatial_data')
#merge location with df on postal code, left mode
df=df.merge(geospatialdata,on='Postal Code',how='left')
df.head(11)

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,9,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,11,M3B,North York,Don Mills,43.745906,-79.352188
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Part 3
